# Contact Lens Color Inspection – Prototype Walkthrough

7개 섹션으로 파이프라인을 단계별 시각화합니다. 아래 셀을 순서대로 실행하세요.

In [ ]:
# 1. 환경 설정 - 모듈 임포트
from pathlib import Path
import sys
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.style.use("seaborn-v0_8")

# 프로젝트 루트 경로 정렬 (notebooks 디렉터리 내부 실행 대비)
PROJECT_ROOT = Path.cwd().resolve()
if PROJECT_ROOT.name == "notebooks":
    PROJECT_ROOT = PROJECT_ROOT.parent
sys.path.insert(0, str(PROJECT_ROOT))

from src.pipeline import InspectionPipeline
from src.utils.file_io import read_json
from src.core.image_loader import ImageLoader
from src.core.lens_detector import LensDetector
from src.core.radial_profiler import RadialProfiler
from src.core.zone_segmenter import ZoneSegmenter
from src.core.color_evaluator import ColorEvaluator

DATA_DIR = PROJECT_ROOT / "data" / "raw_images"
SKU_CONFIG_PATH = PROJECT_ROOT / "config" / "sku_db" / "SKU001.json"
SAMPLE_OK = DATA_DIR / "OK_001.jpg"
SAMPLE_NG = DATA_DIR / "NG_001.jpg"

sku_config = read_json(SKU_CONFIG_PATH)
pipeline = InspectionPipeline(sku_config)

image_loader: ImageLoader = pipeline.image_loader
lens_detector: LensDetector = pipeline.lens_detector
radial_profiler: RadialProfiler = pipeline.radial_profiler
zone_segmenter: ZoneSegmenter = pipeline.zone_segmenter
color_evaluator: ColorEvaluator = pipeline.color_evaluator


def bgr_to_rgb(img: np.ndarray) -> np.ndarray:
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB) if img is not None else img


print(f"PROJECT_ROOT: {PROJECT_ROOT}")
print(f"Loaded SKU config: {sku_config.get('sku_code')}")

## 2. 이미지 로드 (ImageLoader) - 전처리 결과 비교

In [ ]:
raw_image = image_loader.load_from_file(SAMPLE_OK)
processed_image = image_loader.preprocess(raw_image)

fig, axes = plt.subplots(1, 2, figsize=(12, 5))
axes[0].imshow(bgr_to_rgb(raw_image))
axes[0].set_title("원본")
axes[0].axis("off")

axes[1].imshow(bgr_to_rgb(processed_image))
axes[1].set_title("전처리 (ROI + 노이즈 제거 + WB)")
axes[1].axis("off")
plt.show()

## 3. 렌즈 검출 (LensDetector) - 검출 결과 오버레이

In [ ]:
detection = lens_detector.detect(processed_image)
overlay = processed_image.copy()
cv2.circle(overlay, (int(detection.center_x), int(detection.center_y)), int(detection.radius), (0, 255, 0), 2)
cv2.circle(overlay, (int(detection.center_x), int(detection.center_y)), 3, (0, 0, 255), -1)

fig, ax = plt.subplots(1, 1, figsize=(6, 6))
ax.imshow(bgr_to_rgb(overlay))
ax.set_title(f"Lens center=({detection.center_x:.1f}, {detection.center_y:.1f}), r={detection.radius:.1f}")
ax.axis("off")
plt.show()

## 4. 극좌표 변환 (RadialProfiler) - LAB 프로파일 그래프

In [ ]:
radial_profile = radial_profiler.extract_profile(processed_image, detection)

fig, ax = plt.subplots(1, 1, figsize=(8, 5))
ax.plot(radial_profile.r_normalized, radial_profile.L, label="L*")
ax.plot(radial_profile.r_normalized, radial_profile.a, label="a*")
ax.plot(radial_profile.r_normalized, radial_profile.b, label="b*")
ax.set_xlabel("Normalized radius (0=center → 1=outer)")
ax.set_ylabel("LAB value")
ax.set_title("Radial LAB Profile")
ax.legend()
ax.grid(True, alpha=0.3)
plt.show()

## 5. Zone 분할 (ZoneSegmenter) - Zone 경계선 표시

In [ ]:
zones = zone_segmenter.segment(radial_profile)

fig, ax = plt.subplots(1, 1, figsize=(8, 5))
ax.plot(radial_profile.r_normalized, radial_profile.a, label="a* (기준 채널)")

for z in zones:
    ax.axvspan(z.r_end, z.r_start, color="orange" if z.zone_type == "mix" else "skyblue", alpha=0.2)
    ax.text((z.r_start + z.r_end) / 2, ax.get_ylim()[1], z.name, ha="center", va="bottom")

ax.set_xlabel("Normalized radius")
ax.set_ylabel("a*")
ax.set_title("Zone Segmentation")
ax.legend()
ax.grid(True, alpha=0.3)
plt.show()

## 6. 색상 평가 (ColorEvaluator) - ΔE 바 차트

In [ ]:
inspection_result = color_evaluator.evaluate(zones, sku="SKU001", sku_config=sku_config)

zone_names = [zr.zone_name for zr in inspection_result.zone_results]
delta_es = [zr.delta_e for zr in inspection_result.zone_results]
thresholds = [zr.threshold for zr in inspection_result.zone_results]

x = np.arange(len(zone_names))
width = 0.35

fig, ax = plt.subplots(1, 1, figsize=(8, 5))
bars = ax.bar(x - width/2, delta_es, width, label="ΔE (measured)")
ax.bar(x + width/2, thresholds, width, label="Threshold")

ax.set_xticks(x)
ax.set_xticklabels(zone_names)
ax.set_ylabel("ΔE")
ax.set_title(f"Color Evaluation – Judgment: {inspection_result.judgment}")
ax.legend()
ax.grid(True, axis="y", alpha=0.3)

for rect in bars:
    height = rect.get_height()
    ax.annotate(f"{height:.2f}", xy=(rect.get_x() + rect.get_width() / 2, height),
                xytext=(0, 3), textcoords="offset points", ha="center", va="bottom", fontsize=9)

plt.show()
print(f"Overall judgment: {inspection_result.judgment}, ΔE={inspection_result.overall_delta_e:.2f}, confidence={inspection_result.confidence:.2f}")
print("NG reasons:", inspection_result.ng_reasons if inspection_result.ng_reasons else "None")

## 7. 전체 파이프라인 - 10장 배치 처리 및 요약

In [ ]:
batch_paths = sorted(DATA_DIR.glob("*.jpg"))
batch_results = pipeline.process_batch([str(p) for p in batch_paths], sku="SKU001")

summary = pd.DataFrame([
    {
        "image": Path(p).name,
        "sku": r.sku,
        "judgment": r.judgment,
        "overall_delta_e": r.overall_delta_e,
        "confidence": r.confidence
    }
    for r, p in zip(batch_results, batch_paths)
])

display(summary)
print("\n판정 요약:")
print(summary.groupby("judgment").size())
print("\n평균 ΔE:", summary["overall_delta_e"].mean())